In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna

In [2]:
# base_path = '/home/clas_giulia_s/buckets/b1/'
base_path = '/Users/ignacio/MAESTRIA/DMEF/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
# dataset_file = 'competencia_02.parquet'
dataset_file = 'competencia_01_fe_modelito.csv'

ganancia_acierto = 273000
costo_estimulo = 7000

In [3]:
semillas = [5623, 292, 7494, 8504, 1663, 785, 5377, 4838, 2141, 2235, 9836, 1258, 3273, 8349, 1639, 1597, 3195, 40, 5186, 9278, 6281, 7515, 2046, 5642, 505, 4611, 3008, 2063, 2280, 1148, 618, 4806, 1503, 3926, 6363, 400, 2662, 9432, 1632, 386, 2545, 228, 1561, 3523, 4508, 9190, 8181, 7302, 6250, 7762, 8141, 6854, 622, 5327, 6379, 3867, 5420, 3030, 7275, 2040, 6042, 4365, 231, 8330, 8527, 2420, 2558, 9618, 3937, 555, 122, 4907, 7838, 5246, 100, 3243, 1449, 1052, 1906, 7657, 753, 4320, 4576, 9621, 8868, 8155, 7410, 2320, 6355, 1994, 7775, 8358, 3508, 3064, 3904, 3602, 5308, 6947, 1544, 624]

In [ ]:

data = pd.read_csv(dataset_path + dataset_file)
data = data[data["foto_mes"] == 202106] 

In [6]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [7]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [8]:
X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

### Predicciones para Kaggle

Se generan la prediccion para cada modelo individual

In [9]:
version = 'modelito' # UPADTE

modelos = [f'lgb_competencia1_{version}_s{semilla}_final.txt' for semilla in semillas]

In [10]:
len(modelos)

100

In [ ]:
X_test_with_predictions = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()

In [ ]:
counter = 0

for modelo in modelos:
    # Cargamos el modelo
    model = lgb.Booster(model_file=f'{modelos_path}{version}/{modelo}')
    
    # Predecimos para agosto
    X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()
    y_pred_lgm = model.predict(X_test)
    X_test_with_predictions[f'pred_lgm_{semillas[counter]}'] = y_pred_lgm
    
    print(f'{counter}. Predicciones del modelo: {modelo} DONE')
    
    # Generamos la columna 'Predicted'
    envios = np.zeros(len(X_test_with_predictions), dtype=int)
    envios[:100] = 1 
    
    # Asignamos 'envios' a la columna 'Predicted'
    X_test_with_predictions['Predicted'] = envios
    
    # Creamos el dataframe para guardar
    output = X_test_with_predictions[['numero_de_cliente', 'Predicted']]
    
    # Definimos el nombre y la ruta del archivo
    file_name = f'results_unit_{version}_{counter}.csv'
    output_path = base_path + f'exp/competencia_1/{version}/' + file_name
    
    # Guardamos el archivo CSV
    output.to_csv(output_path, index=False)
    print(f'Archivo guardado en: {output_path}')
    
    counter += 1

0. Predicciones del modelo: lgb_competencia1_modelito_s5623_final.txt DONE
Archivo guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_unit_modelito_0.csv


/var/folders/6c/1g3b2t8s1l5ch_c9nch8gppw0000gn/T/ipykernel_27438/2842301844.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_with_predictions['Predicted'] = envios


1. Predicciones del modelo: lgb_competencia1_modelito_s292_final.txt DONE
Archivo guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_unit_modelito_1.csv
2. Predicciones del modelo: lgb_competencia1_modelito_s7494_final.txt DONE
Archivo guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_unit_modelito_2.csv
3. Predicciones del modelo: lgb_competencia1_modelito_s8504_final.txt DONE
Archivo guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_unit_modelito_3.csv
4. Predicciones del modelo: lgb_competencia1_modelito_s1663_final.txt DONE
Archivo guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_unit_modelito_4.csv
5. Predicciones del modelo: lgb_competencia1_modelito_s785_final.txt DONE
Archivo guardado en: /Users/ignacio/MAESTRIA/DMEF/exp/competencia_1/modelito/results_unit_modelito_5.csv
6. Predicciones del modelo: lgb_competencia1_modelito_s5377_final.txt DONE
Archivo guardado en: /Users